In [30]:
import os
from pathlib import Path
# Check if the code is running on Google Colab
try:
    import google.colab
    IN_COLAB = True
    base_path = "/content/"
    if Path(f"{base_path}final_project").is_dir():
      %cd {base_path}final_project
      !git pull
      %cd {base_path}
    else:
      !git clone https://github.com/fernandaluft/final_project.git
except ImportError:
    IN_COLAB = False
    base_path = "/workspaces/"

/content/final_project
Already up to date.
/content


In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from final_project.src.scraping import Scraping
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
import pickle
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [47]:
scraping = Scraping(IN_COLAB, sentiment_ds = True)
scraping.kaggle_scrape()

In [52]:
class SentimentMLTrain():
  def __init__(self, dataset_limit):
    self.dataset_limit = dataset_limit
    self.stop_words = set(stopwords.words('english'))
    self.lemmatizer = WordNetLemmatizer()
    self.tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 3))
    self.X_train, self.X_test, self.y_train, self.y_test = None, None, None, None
    self.best_model = None
    with open('/content/final_project/models/tf_idf.pickle', 'wb') as f:
      pickle.dump(self.tfidf_vectorizer, f)


  def read_sentiment_dataset(self):
    !unzip -o -n /content/imdb-dataset-of-65k-movie-reviews-and-translation.zip -d {base_path}final_project/data
    os.system(f'rm -rf /content/imdb-dataset-of-65k-movie-reviews-and-translation.zip')
    self.sentiment_df = pd.read_csv(f"{base_path}final_project/data/IMDB-Dataset.csv").sample(1000)

  def preprocess_text(self, text):
    text = re.sub(r'\W|\d', ' ', str(text))  # Remove special characters and digits
    tokens = word_tokenize(text.lower())  # Tokenize and convert to lowercase
    lemmatized_tokens = [self.lemmatizer.lemmatize(word) for word in tokens if word not in self.stop_words]  # Lemmatize and remove stopwords
    return ' '.join(lemmatized_tokens)

  def preprocess_data(self):
    self.sentiment_df['clean_text'] = self.sentiment_df['Reviews'].apply(self.preprocess_text)

  def split_data(self):
    self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
        self.tfidf_vectorizer.fit_transform(self.sentiment_df['clean_text']),
        self.sentiment_df['Ratings'],
        test_size=0.2,
        random_state=42
    )

  def train_model(self):
    svm_classifier = SVC(kernel='linear')
    param_grid = {'C': [0.1, 1, 10, 100]}  # Hyperparameter grid for tuning
    grid_search = GridSearchCV(svm_classifier, param_grid, cv=5)
    grid_search.fit(self.X_train, self.y_train)
    self.best_model = grid_search.best_estimator_

  def evaluate_model(self):
    y_pred = self.best_model.predict(self.X_test)
    accuracy = accuracy_score(self.y_test, y_pred)
    print("Accuracy:", accuracy)
    print(classification_report(self.y_test, y_pred))

  def save_model(self, model_path):
    with open(model_path, 'wb') as f:
      pickle.dump(self.tfidf_vectorizer, f)



In [53]:
sentiment_ml_train = SentimentMLTrain(None)
sentiment_ml_train.read_sentiment_dataset()

caution:  both -n and -o specified; ignoring -o
Archive:  /content/imdb-dataset-of-65k-movie-reviews-and-translation.zip


In [54]:
sentiment_ml_train.sentiment_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 96255 to 147162
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Ratings   1000 non-null   float64
 1   Reviews   1000 non-null   object 
 2   Movies    1000 non-null   object 
 3   Resenhas  1000 non-null   object 
dtypes: float64(1), object(3)
memory usage: 39.1+ KB


In [55]:
sentiment_ml_train.preprocess_data()
sentiment_ml_train.split_data()
sentiment_ml_train.train_model()
sentiment_ml_train.evaluate_model()

Accuracy: 0.235
              precision    recall  f1-score   support

         1.0       0.38      0.32      0.34        19
         2.0       0.09      0.08      0.09        12
         3.0       0.17      0.33      0.23        18
         4.0       0.11      0.09      0.10        23
         5.0       0.16      0.31      0.21        16
         6.0       0.44      0.17      0.24        24
         7.0       0.31      0.12      0.18        32
         8.0       0.20      0.18      0.19        17
         9.0       0.30      0.35      0.32        17
        10.0       0.33      0.45      0.38        22

    accuracy                           0.23       200
   macro avg       0.25      0.24      0.23       200
weighted avg       0.26      0.23      0.23       200



In [56]:
sentiment_ml_train.save_model("/content/final_project/models/model1k.pkl")

In [38]:
!unzip -o -n /content/final_project/preprocessed_data/xaa_books_reviews.zip -d {base_path}final_project/data

caution:  both -n and -o specified; ignoring -o
Archive:  /content/final_project/preprocessed_data/xaa_books_reviews.zip


In [59]:
import pickle
def calculate_sentiment_book(title):
  sentiment_ml = SentimentMLTrain(None)
  n_neg = 0
  n_pos = 0
  with open("/content/final_project/models/model1k.pkl", "rb") as f:
    model = pickle.load(f)

  with open("/content/final_project/models/tf_idf.pickle", "rb") as f:
    vec = pickle.load(f)
  books = pd.read_csv("/content/final_project/data/content/final_project/data/books_reviews.csv")
  books_subset = books[books.Title == title]['review/text']
  books_subset.info()
  for rev, rev2 in books_subset.iteritems():
    processed = sentiment_ml.preprocess_text(rev2)
    processed = vec.transform(rev2)
    score = model.predict(processed)
    if score >= 5:
      n_pos += 1
    else:
      n_neg += 1

  return [n_neg, n_pos]

In [60]:
print(calculate_sentiment_book('Run Baby Run'))

<class 'pandas.core.series.Series'>
Int64Index: 4 entries, 0 to 42256
Series name: review/text
Non-Null Count  Dtype 
--------------  ----- 
4 non-null      object
dtypes: object(1)
memory usage: 64.0+ bytes


<ipython-input-59-6c5ac5f93ac6>:14: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for rev, rev2 in books_subset.iteritems():


NotFittedError: The TF-IDF vectorizer is not fitted